# Function Codegen

SymForce provides flexible tools to generate functions and classes from symbolic expressions, that can be mixed with handwritten code or built up into standalone autogenerated executable packages.

We can generate executable code from a symbolic function in two lines using the `FunctionCodegen` class. The underlying mechanism is to have code generated functions created directly from symbolic counterparts by running through the function with symbolic quantities, then getting some extra metadata to guide naming and types.

## Setup

In [ ]:
# Setup
import symforce
symforce.set_backend('symengine')
symforce.set_log_level('warning')

from symforce.notebook_util import display, display_code

In [ ]:
from symforce import sympy as sm
from symforce import geo

from symforce.codegen import CodegenMode
from symforce.codegen import FunctionCodegen

In [ ]:
def az_el_from_point(nav_T_cam, nav_t_point, epsilon=0):
    """
    Transform a nav point into azimuth / elevation angles in the
    camera frame.

    Args:
        nav_T_cam (geo.Pose3): camera pose in the world
        nav_t_point (geo.Matrix): nav point
        epsilon (Scalar): small number to avoid singularities

    Returns:
        geo.Matrix: (azimuth, elevation)
    """
    cam_t_point = nav_T_cam.inverse() * nav_t_point
    x, y, z = cam_t_point
    theta = sm.atan2(y, x + epsilon)
    phi = sm.pi / 2 - sm.acos(z / (cam_t_point.norm() + epsilon))
    return geo.V2(theta, phi)

In [ ]:
spec = FunctionCodegen(
    name='AzElFromPoint',
    func=az_el_from_point,
    arg_types=[geo.Pose3, geo.V3(), sm.Symbol],
    return_type=geo.V2()
)

display_code(spec.render(mode=CodegenMode.CPP), 'C++')

In [ ]:
nav_T_cam = geo.Pose3(
    R=geo.Rot3.symbolic('q'),
    t=geo.V3().symbolic('t')
)

nav_t_point = geo.V3().symbolic('P')

az_el = az_el_from_point(
    nav_T_cam=nav_T_cam,
    nav_t_point=nav_t_point,
    epsilon=sm.Symbol('epsilon')
)
display(az_el)

In [ ]:
result = az_el.jacobian(geo.M(nav_T_cam.to_storage()))

In [ ]:
def az_el_from_point_derivative_nav_T_cam(nav_T_cam, nav_t_point, epsilon):
    az_el = az_el_from_point(nav_T_cam, nav_t_point, epsilon)
    return az_el.jacobian(geo.M(nav_T_cam.to_storage()))

spec = FunctionCodegen(
    name='AzElFromPoint_Derivative_nav_T_cam',
    func=az_el_from_point_derivative_nav_T_cam,
    arg_types=[geo.Pose3, geo.V3(), sm.Symbol],
    return_type=geo.M.zeros(2, 7)
)

display_code(spec.render(mode=CodegenMode.CPP), 'C++')